<a href="https://www.kaggle.com/code/sjagkoo7/bmi-body-mass-index-prediction?scriptVersionId=143185517" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Introduction
In this case study , we will be working with a dataset that contains information about individual's height,weight,gender,and body mass index(BMI). The BMI is a Measure that uses your height and weight to work out if your weight is healthy. The BMI calculation divides an adult's weight in kilograms by their height in meters squared.
The dataset contains the followings columns:

* `Gender : The gender of the individual`
* `Height : The height of the individual in cm`
* `Weight : The weight of individual in kg`
* `Index : The BMI index of the individual , categorized as follows:`

   `0:Extremly Weak
    1:Weak
    2:Normal
    3:Overweight
    4:Obesity
    5:Extremly Obesity`

# Importing Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for visualization like pie graph
import seaborn as sns # for visualization like scatter plot 
from ydata_profiling import profile_report

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Reading the Dataset

In [ ]:
#reading the train & validation data set
train=pd.read_csv('/kaggle/input/bmi-body-mass-index/bmi_train.csv')
validation=pd.read_csv('/kaggle/input/bmi-body-mass-index/bmi_validation.csv')

In [ ]:
#displaying first 2 rows train dataset
train.head(2)

In [ ]:
#displaying first 2 rows validation dataset
validation.head(2)

# Exploring the Dataset

In [ ]:
# train dataset
# train.shape # total rows & columns
# len(train) # total rows in dataset
# train.size  # total elements in dataset
train.info()

In [ ]:
# validation dataset
# validation.shape # total rows & columns
# len(validation) # total rows in dataset
# validation.size  # total elements in dataset
validation.info()

### `**in this train/validation dataset there is one categorical feature. Hence it will require feature encoding**`

In [ ]:
#unique values on train dataset
#train.nunique()
#unique values on train dataset
# validation.nunique()

In [ ]:
#train dataset - displaying rows if any have duplicate rows
train_rows_data=train[train.duplicated()]
train_rows_data

### `**There are duplicates in train dataset.Hence it will require to remove the duplicates**`

In [ ]:
#validation dataset - displaying rows if any have duplicate rows
validation_rows_data=validation[validation.duplicated()]
validation_rows_data

### `**There are no duplicates in validation dataset**`

In [ ]:
#null values in train dataset
#train.isna().sum()
#null values in validation dataset
#train.isna().sum()

In [ ]:
#train dataset - displaying rows if any have null rows
train[train.isna().any(axis=1)]

In [ ]:
#validation dataset - displaying rows if any have null rows
validation[validation.isna().any(axis=1)]

### `**There is no any null values in train and validation dataset**`

In [ ]:
# statatics summary train dataset
#train.describe()
# statatics summary validation dataset
#validation.describe()

In [ ]:
# train dataset
train_summary=pd.DataFrame(index=train.columns)
train_summary['count']=train.count()
train_summary['nunique']=train.nunique()
train_summary['null']=train.isna().sum()
train_summary['dtypes']=train.dtypes
train_summary=pd.concat([train_summary,train.describe().T.drop('count',axis=1)],axis=1)
train_summary

In [ ]:
# validation dataset
validation_summary=pd.DataFrame(index=validation.columns)
validation_summary['count']=validation.count()
validation_summary['nunique']=validation.nunique()
validation_summary['null']=validation.isna().sum()
validation_summary['dtypes']=validation.dtypes
validation_summary=pd.concat([validation_summary,validation.describe().T.drop('count',axis=1)],axis=1)
validation_summary

In [ ]:
# detail report - dasborad train dataset
train.profile_report(title ='detail report - dasborad train dataset',progress_bar=False)

In [ ]:
# detail report - dasborad validation dataset
validation.profile_report(title ='detail report - dasborad validation dataset',progress_bar=False)

In [ ]:
train_t=train.copy()
validation_v=validation.copy()

# Data Cleaning

In [ ]:
#removing duplicate from train datset
train_t.drop_duplicates(inplace=True)

In [ ]:
train_t[train_t.duplicated()]

# Feature Encoding

In [ ]:
# Feature encoding on Train dataset
train_t['Gender']=train_t['Gender'].map({'Male':0 ,'Female':1})
# Feature encoding on  validation dataset
validation_v['Gender']=validation_v['Gender'].map({'Male':0 ,'Female':1})

In [ ]:
train_t.head(2)

In [ ]:
validation_v.head(2)

# Co-Realtion Matrix

In [ ]:
#Co-relation train data set
corr=train_t.corr()
corr

In [ ]:
plt.figure(figsize=(4,2))
sns.heatmap(corr,cmap='viridis',annot=True,fmt='0.2f')
plt.title('Co-relation of training dataset\n',fontsize='12',fontweight='bold')

In [ ]:
#Co-relation validation data set
corr=validation_v.corr()
corr

In [ ]:
plt.figure(figsize=(4,2))
sns.heatmap(corr,cmap='YlGnBu',annot=True,fmt='0.2f')
plt.title('Co-relation of validation dataset\n',fontsize='12',fontweight='bold')

### `**weight is highly co-realted with index**`

# Gender Effect on Index

In [ ]:
plt.figure(figsize=(8,5))
g=sns.countplot(data=train_t,x='Gender',hue='Index',palette='viridis') # palette='viridis'-- for coloring
plt.title('Gender Effect on Index\n',fontsize=12,fontweight='bold')
g.set_xticklabels(['Male','Female'])

### `** Male is having more Obesity/Index as comapre to Female**`

# Height Effect on Gender

In [ ]:
plt.figure(figsize=(8,5))
g=sns.boxplot(data=train_t,x='Gender',y='Height',palette='Set2') # palette='Set2'-- for coloring
plt.title('Height Effect on Gender\n',fontdict={'fontsize':12,'fontweight':'bold'})
g.set_xticklabels(['Male','Female'])

### `**graph shows the  height of male is less than feamle usually**`

# Weight Effect on Gender

In [ ]:
plt.figure(figsize=(8,5))
g=sns.boxplot(data=train_t,x='Gender',y='Weight',palette='viridis') # palette='viridis'-- for coloring
plt.title('Height Effect on Gender\n',fontdict={'fontsize':12,'fontweight':'bold'})
g.set_xticklabels(['Male','Female'])

### `**graph shows the weight of female is less than male usually**`

# Height & Weight Effect on Index

In [ ]:
plt.figure(figsize=(8,5))
sns.scatterplot(data=train_t,x='Height',y='Weight',hue='Index',palette='viridis') # palette='viridis'-- for coloring
plt.title('Height & Weight Effect on Index\n',fontdict={'fontsize':12,'fontweight':'bold'})

### `**graph indicates there are more Obesity/Index than normal health condtion**`

In [ ]:
plt.figure(figsize=(10,8))
sns.pairplot(data=train_t,hue='Index',palette='viridis',kind='scatter') # palette='viridis'-- for coloring
plt.suptitle('Pair Plot of Numerical Variables\n')
plt.tight_layout()